# Running a Variational Quantum Algorithm on IQM Crystal using Qrisp

This notebook demonstrates how to run a variational quantum algorithm on an IQM Crystal quantum processor using the Qrisp library. The example uses the Variational Quantum Eigensolver (VQE) to calculate the ground state energy of the Hydrogen molecule (H2).

In order to install Qrisp, you can use the following command:


In [ ]:
!pip install "qrisp[iqm]"


## Define an IQM Backend

Then, we need to define an `IQMBackend`:

In [ ]:
from qrisp.interface import IQMBackend
qrisp_garnet = IQMBackend(api_token = "YOUR_TOKEN", device_instance = "garnet")

## Using VQE to calculate the ground state energy of the Hydrogen molecule on IQM Garnet
Qrisp has built-in functionality to run VQE on an IQM backend. The following code demonstrates how to do this:

In [7]:
from qrisp import *
from qrisp.operators.qubit import X,Y,Z

# Problem Hamiltonian
c = [-0.81054, 0.16614, 0.16892, 0.17218, -0.22573, 0.12091, 0.166145, 0.04523]
H = c[0] \
    + c[1]*Z(0)*Z(2) \
    + c[2]*Z(1)*Z(3) \
    + c[3]*(Z(3) + Z(1)) \
    + c[4]*(Z(2) + Z(0)) \
    + c[5]*(Z(2)*Z(3) + Z(0)*Z(1)) \
    + c[6]*(Z(0)*Z(3) + Z(1)*Z(2)) \
    + c[7]*(Y(0)*Y(1)*Y(2)*Y(3) + X(0)*X(1)*Y(2)*Y(3) + Y(0)*Y(1)*X(2)*X(3) + X(0)*X(1)*X(2)*X(3))

# Ansatz
def ansatz(qv,theta):
    for i in range(4):
        ry(theta[i],qv[i])
    for i in range(3):
        cx(qv[i],qv[i+1])
    cx(qv[3],qv[0])

from qrisp.vqe.vqe_problem import *

vqe = VQEProblem(hamiltonian = H,
                 ansatz_function = ansatz,
                 num_params=4,
                 callback=True)

energy = vqe.run(qarg = QuantumVariable(4),
              depth = 1,
              max_iter=30,
              #mes_kwargs={'precision':0.01,'diagonalisation_method':'commuting'} # RUN ON SIMULATOR
              mes_kwargs={'backend':qrisp_garnet,'precision':0.02,'diagonalisation_method':'commuting'} # RUN ON IQM Garnet 
            )
print(energy)
# Yields -1.864179046

-1.6314065797709651


In [ ]:

# Copyright 2025 (Rene Zander, Stefan Seegerer)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.